
## VI DU 9.17-CHUONG 9,  TRANG 370
# CAC BIEN: hours= số giờ làm việc được trả công ; wage= tiền công/giờ;age= số tuổi;edu= số năm được đào tạo; kidslt6= số con có tuổi <6; nwifeinc= thu nhập ngoài tiền công (bao gồm cả thu nhập của chồng); exper= số năm kinh nghiệm. Data: mroz.txt
# He phuong trinh tuyến tính : hours-f(ln(wage), edu, age, didslt6,nwifeinc); Ln(wage)=g(hours, edu, exper, exper^2)
# Call packages for estimating, test


In [9]:
library(foreign)
library(car); library(carData)
library(lmtest);library(zoo)
library(sandwich)
library(stargazer)
library(caTools)
library(caret)
library(ivreg)


Warning message:
"package 'caret' was built under R version 4.2.3"
Loading required package: ggplot2

Warning message:
"package 'ggplot2' was built under R version 4.2.3"
Error: package or namespace load failed for 'ggplot2' in loadNamespace(i, c(lib.loc, .libPaths()), versionCheck = vI[[i]]):
 namespace 'rlang' 1.0.6 is already loaded, but >= 1.1.0 is required



ERROR: Error: package 'ggplot2' could not be loaded


In [11]:
#mroz=read_excel("D:/dataR/ch9/mroz.xls")
mroz=read.table("mroz.txt", header=TRUE)
# Chỉ lấy các biến dùng trong bài để tiết kiệm thời gian xử lý
data=mroz%>%select(hours, lwage, age,educ,exper,nwifeinc,kidslt6,kidsge6)
# Xem tên biến và số liệu ở 2 dòng đầu
head(data,2)
# Chia data thành hai tập train và test, với tỷ lệ 80:20
indexTrain=sample.split(data,SplitRatio =0.8)
train.set=subset(data, indexTrain == TRUE)
head(train.set,2)
test.set= subset(data, indexTrain == FALSE)
hours=train.set$hours
lwage=train.set$lwage
educ=train.set$educ
age=train.set$age
kidslt6=train.set$kidslt6
nwifeinc=train.set$nwifeinc
exper=train.set$exper
# plot(age)

ERROR: Error in mroz %>% select(hours, lwage, age, educ, exper, nwifeinc, kidslt6, : could not find function "%>%"


## PHƯƠNG PHÁP 2SLS REGRESSION

## UOC LUONG HE PHUONG TRINH - JOINT ESTIMATION OF SYSTEM

In [ ]:
# Ưu điểm của cách này là các biến công cụ ở mỗi phowng trình có thể thêm/bớt
reg5=ivreg(hours~lwage+educ+age+kidslt6+nwifeinc|educ+age+kidslt6+nwifeinc+poly(exper,2),data=train.set)
reg6=ivreg(lwage~hours+educ+poly(exper,2)|educ+age+kidslt6+nwifeinc+poly(exper,2),data=train.set)
stargazer(reg5,reg6, type="text") 

In [ ]:
library(systemfit)
# DINH NGHIA HE PHUONG TRINH
eq.hours=hours~lwage+educ+age+kidslt6+nwifeinc
eq.lwage=lwage~hours+educ+poly(exper,2) 
eq.system=list(eq.hours, eq.lwage)
instrum=~educ+poly(age,2)+kidslt6+nwifeinc+poly(exper,2)
my.sys=systemfit(eq.system,inst=instrum,method="2SLS",data=train.set)
summary(my.sys)
hourshat=predict(my.sys$eq[[1]])
head(hourshat,2)

In [ ]:
library(forecast)
test.system=predict(my.sys,newdata=test.set,se.fit = FALSE, se.pred = FALSE,
                             interval = "confidence", level=0.95)

head(test.system,3)

print("Đánh giá dự báo hours trên test.set")
accuracy(test.system$eq1.pred,test.set$hours)
print("Đánh giá dự báo lwage trên test.set")
accuracy(test.system$eq2.pred,test.set$lwage)

## LẤY RA VEC TƠ HỆ SỐ CỦA TỪNG PHƯƠNG TRÌNH, VÉC TƠ HỆ SỐ CÁC BIẾN NGOẠI SINH, MA TRẬN HỆ SỐ BIẾN NÔI SINH      

In [ ]:
betahat=coef(my.sys)
betahat
betahat_eq1=coef(my.sys$eq[[1]])
print('Vec tơ beta phương trình 1')
betahat_eq1
print('Vec tơ beta các biến ngoại sinh phương trình 1')
free_b1=betahat_eq1[-1]   
free_b1
print('Vec tơ beta phương trình 2')     
betahat_eq2=coef(my.sys$eq[[2]])
betahat_eq2
print('Vec tơ beta các biến ngoại sinh phương trình 2')
free_b2=betahat_eq2[-1]
free_b2

## DỰ BÁO CÁC BIẾN PHỤ THUỘC TẠI TRUNG BÌNH CỦA CÁC BIẾN

In [ ]:
# CẦN GIẢI HỆ PHOWNG TRÌNH TUYẾN TÍNH, Ax=b
i=1
educ=mean(educ)
age=mean(age)
kidslt6=mean(kidslt6)
nwifeinc=mean(nwifeinc)
exper=mean(exper)

# Tính véc tư b
b1=-(i*free_b1[1]+educ*free_b1[2]+age*free_b1[3]+kidslt6*free_b1[3]+nwifeinc*free_b1[4])

b1
b2=-(i*free_b2[1]+educ*free_b2[2]+exper*free_b2[3]+exper^2*free_b2[4])
b2
b=cbind(b1,b2)
print('Véc tơ b')
b

#  Xây dựng ma trận A

c1=c(1,-free_b2[1])
c1
c2=c(-free_b1[1],1)
c2
A=cbind(c1,c2)
print('Ma trận A')
A
# Giải hệ phowng trình
output=solve(A,t(b))
output